<a href="https://colab.research.google.com/github/avdlaan/group4/blob/master/Simple_QA_Read_Data_functions_from_Data_set_with_Ktrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip3 install ktrain==0.29.2
!pip install tensorflow==2.4.1

In [ ]:
import numpy as np
import pandas as pd 
import glob
import json
import math

Set the initial imports- for the next step let us iterate the directory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

all_json = glob.glob('/content/drive/MyDrive/train/document_parses/json_test/*.json', recursive=True)

# let us get the list of articles
len(all_json)
print(len(all_json))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
39


In [ ]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:400]}... {self.body_text[:500]}...'

In [ ]:
first_row = FileReader(all_json[0])
print(first_row)

000a0fc8bbef80410199e690191dc3076a290117: Background: Various transcription factors are involved in the process of mutually exclusive expression and clonal variation of the Plasmodium multigene (var) family. Recent studies revealed that a P. falciparum SWI/SNF-related matrix-associated actin-dependent regulator of chromatin (PfSWIB) might trigger stage-specific programmed cell death (PCD), and was not only crucial for the survival and dev... Plasmodium falciparum, which causes malignant malaria such as cerebral malaria (CM) or pregnancy-associated malaria (PAM) [1, 2] , is the cause of death to 435,000 individuals annually [3] . Plasmodium falciparum erythrocyte membrane protein 1 (PfEMP1), which is encoded by the var gene family comprising approximately 60 members, is the major virulence factor involved in the antigenic variation and clinical pathogenicity of falciparum malaria [4, 5] . It is remarkable that the expression of var f...


In [ ]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': []}
for idx, entry in enumerate(all_json):
    try:
        content = FileReader(entry)
        if (idx % 1000 == 0):
            print(f"processing {idx} of {len(all_json)} ")
        dict_['paper_id'].append(content.paper_id)
        dict_['abstract'].append(content.abstract)
        dict_['body_text'].append(content.body_text)
    except:
        pass
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text'])
#df_covid = pd.DataFrame(dict_, columns=['body_text'])
docs = df_covid['body_text'].values.tolist()
#docs = df_covid.values.tolist()
df_covid.head(39)
print(docs)

processing 0 of 39 
['Plasmodium falciparum, which causes malignant malaria such as cerebral malaria (CM) or pregnancy-associated malaria (PAM) [1, 2] , is the cause of death to 435,000 individuals annually [3] . Plasmodium falciparum erythrocyte membrane protein 1 (PfEMP1), which is encoded by the var gene family comprising approximately 60 members, is the major virulence factor involved in the antigenic variation and clinical pathogenicity of falciparum malaria [4, 5] . It is remarkable that the expression of var family members is mutually exclusive. For each parasite at a time, only one or a few var genes are expressed, while the remaining members are silenced [6, 7] . In each generation, P. falciparum is able to express different var genes, and the switches in var gene expression lead to antigenic variation of PfEMP1, which results in immune evasion and chronic infection [8] [9] [10] [11] . This process has proven to be mediated by epigenetic mechanisms, including chromatin modific

In [ ]:
import ktrain
from ktrain.text.qa import SimpleQA

INDEXDIR = '/content/drive/MyDrive/train/myindex'

SimpleQA.initialize_index(INDEXDIR)
SimpleQA.index_from_list(
    docs, INDEXDIR, commit_every=len(docs),
    limitmb=2048,
    multisegment=True, procs=4, # these args speed up indexing
    breakup_docs=True         # this slows indexing but speeds up answer retrieval
)

qa = SimpleQA(INDEXDIR)

#from ktrain.text.qa import AnswerExtractor

#ae = AnswerExtractor()

#df_covid = ae.extract(df_covid.body_text.values, df_covid, [('What are the proteins in Covid Virus?', 'Proteins'),
 #                                    ('Effects of Covid?','Effects'),('What the Covid Virus Strains?','Strains')])

#df_covid.head(39)

In [ ]:

answers = qa.ask('Which are the proteins in the Covid Virus?')
qa.display_answers(answers[:5])

,Candidate Answer,Context,Confidence,Document Reference
0,a nucleocapsid protein,"feline infectious peritonitis virus (fipv) is a member of the family coronaviridae and is a single stranded rna virus with three major structural proteins, including a nucleocapsid protein (lai, 1990).",0.997210,23
1,"? to respond effectively to the recent severe acute respiratory syndrome coronavirus 2 (sars co v 2) pandemic, it is essential to understand the molecular basis for a successful immune response to coronavirus infection (iwasaki and yang (2020)). in particular, characterising the b cell response is important as the identification of potent neutralising antibodies could pave the way for effective treatments, aid in prior exposure diagnosis, or assist in predicting vaccine efficacy (e. g. tay et al. (2020) ; liao et al. (2020) ; wen et al. (2020) ; zhao et al. (2020) ; nielsen et al. (2020) ; galson et al. (2020)). such molecular characterisations of binding / neutralising antibodies to sars co v 2 antigens are now beginning to emerge. early studies have shown a large proportion of the sars co v 2 neutralising response is sars co v 2 specific, although some sars co v 2 / sars co v 1 (the virus responsible for the 2003 epidemic) crossneutralising antibodies do exist (pinto et al. (2020) ; yuan et al. (2020) ; robbiani et al. (2020)). this is possible because many of the sars co v 1 and sars co v 2 spike protein domains","it will save valuable time in the ? to respond effectively to the recent severe acute respiratory syndrome coronavirus 2 (sars co v 2) pandemic, it is essential to understand the molecular basis for a successful immune response to coronavirus infection (iwasaki and yang (2020)). in particular, characterising the b cell response is important as the identification of potent neutralising antibodies could pave the way for effective treatments, aid in prior exposure diagnosis, or assist in predicting vaccine efficacy (e. g. tay et al. (2020) ; liao et al. (2020) ; wen et al. (2020) ; zhao et al. (2020) ; nielsen et al. (2020) ; galson et al. (2020)). such molecular characterisations of binding / neutralising antibodies to sars co v 2 antigens are now beginning to emerge. early studies have shown a large proportion of the sars co v 2 neutralising response is sars co v 2 specific, although some sars co v 2 / sars co v 1 (the virus responsible for the 2003 epidemic) crossneutralising antibodies do exist (pinto et al. (2020) ; yuan et al. (2020) ; robbiani et al. (2020)). this is possible because many of the sars co v 1 and sars co v 2 spike protein domains",0.001836,9
2,"? the world was made aware of a newly discovered coronavirus via an email from dr. ali mohamed zaki, an egyptian virologist working at the dr. soliman fakeeh hospital in jeddah in the kingdom of saudi arabia (ksa). the email was published on the website of the professional emerging diseases (pro med) network on 20-september 2014 (pro med, 2014. that first case was a 60 year old man from bisha in the ksa and, thanks to the email, the rapid discovery of a second case of the virus, this time in an ill patient from qatar, was transferred to the united kingdom for care (fig. 1) (bermingham et al., 2012). as of 20th january 2015, there have been 969 detections of viral rna or virus specific antibodies","? the world was made aware of a newly discovered coronavirus via an email from dr. ali mohamed zaki, an egyptian virologist working at the dr. soliman fakeeh hospital in jeddah in the kingdom of saudi arabia (ksa). the email was published on the website of the professional emerging diseases (pro med) network on 20-september 2014 (pro med, 2014. that first case was a 60 year old man from bisha in the ksa and, thanks to the email, the rapid discovery of a second case of the virus, this time in an ill patient from qatar, was transferred to the united kingdom for care (fig. 1) (bermingham et al., 2012). as of 20th january 2015, there have been 969 detections of viral rna 

In [ ]:
#df_covid = ae.extract(df_covid.body_text.values, df_covid, [('Covid Recovery?', 'Recovery'),
#                                     ('Long Covid signs?','Signs')])
#df_covid.head(12)